In [100]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.impute import SimpleImputer
import os
import random

In [101]:
def read_files(dir, type):
    # This function gathers all filenames from a directory and puts them in a list.
    #
    # Inputs:
    #   - Full source directory in string format
    # Outputs:
    #   - DataFrame with all files concatenated together

    y = []
    for file in os.listdir(dir):
        f = os.path.join(dir, file)

        if os.path.isfile(f):
            y.append(f)

    df_out = pd.DataFrame(columns=['playerid'])
    if type == 'years':
        for file in y:
            df_working = pd.read_csv(file)
            df_working['year'] = file[-8:-4]

            df_out = pd.concat([df_out, df_working], join='outer')
    elif type == 'pos':
        for file in y:
            df_working = pd.read_csv(file)
            df_working[file[-6:-4]] = True

            df_out = df_out.merge(df_working, on='playerid', how='outer', suffixes=(None, 'yy'))
            df_out.drop(columns=list(df_out.filter(regex='yy').columns), axis=1, inplace=True)

        df_out.drop(columns=['Name', 'Team', 'G'], inplace=True)

    return df_out

In [102]:
def clean_data(df_in, type):
    df = df_in.copy(deep=True)

    if type == 'B':
        df.drop(columns=['Dol', 'Name', 'Age Rng', 'Team'], inplace=True)
        df = df.loc[df['AB'] >= 200]
        df.drop(columns=['AB'], inplace=True)
        df = df.loc[:, ~df.columns.str.contains('1')]
    elif type == 'SP':
        df.drop(columns=['Dollars', 'Name', 'SV', 'Age Rng', 'Team'], inplace=True)
        df = df.loc[df['IP'] >= 100]
        df.drop(columns=['IP'], inplace=True)
        df = df.loc[:, ~df.columns.str.contains('1')]
    elif type == 'SP_Skills':
        df.drop(columns=['Name', 'Team', 'Age'], inplace=True)
        df = df.loc[df['IP'] >= 100]
        df.drop(columns=['IP'], inplace=True)
        df = df.loc[:, ~df.columns.str.contains('1')]
    elif type == 'B_Skills':
        df.drop(columns=['Name', 'Team', 'Age'], inplace=True)
        df = df.loc[df['AB'] >= 200]
        df.drop(columns=['AB'], inplace=True)
        df = df.loc[:, ~df.columns.str.contains('1')]
    else:
        return print("Unknown type selected.")

    # Clear duplicate columns from merge
    for column in df.columns:
        lastchar = column[-3:]
        if lastchar == '_yy':
            df.drop(columns=column, inplace=True)
        elif lastchar == '_xx':
            df.rename(columns={column: column[:-3]}, inplace=True)
        elif column.find('Unnamed') >= 0:
            df.drop(columns=column, inplace=True)
        else:
            pass

    # Clear columns with > 90% of null values and fill the rest with 0
    df = df.dropna(axis=1, thresh=(0.90*df.shape[0]))
    df.fillna(value=0, inplace=True)

    # Convert Year column to int
    df['year'] = df['year'].astype(int)

    # Convert strings with % to floats
    for column in df.columns:
        try:
            df[column] = df[column].str.rstrip('%').astype('float') / 100
        except:
            pass

    for column in df.select_dtypes('object').columns:
        try:
            df[column] = df[column].astype(int)
            df.drop(columns=[column], inplace=True)
        except:
            pass

    # Sort by player and then by year
    df.sort_values(['playerid', 'year'], inplace=True)

    return df

In [103]:
def standardize(df_in):
    df = df_in.copy(deep=True)

    df.reset_index(inplace=True, drop=True)

    features = df.drop(columns=['year', 'playerid']).values

    # imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # features = imputer.fit_transform(features)

    st_scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
    features_scaled = st_scaler.fit_transform(features)

    df_out = pd.DataFrame(data=features_scaled, columns=df.drop(columns=['year', 'playerid']).columns)    
    df_out['playerid'] = df['playerid']
    df_out['year'] = df['year']

    return df_out

In [104]:
def dummies(df_in, params):
    df = df_in.copy(deep=True)

    df = pd.get_dummies(data=df, columns=params, dtype=int)

    return df

In [105]:
def calc_target(df_in, type):
    df = df_in.copy(deep=True)

    # Calculate average result of all output categories, which have already been standardized
    if type == 'SP':
        df['ERA'] = df['ERA'] * -1
        df['WHIP'] = df['WHIP'] * -1 
        df['Target'] = (df[['W', 'ERA', 'SO', 'WHIP']].sum(axis=1))
    elif type == 'B':
        df['Target'] = (df[['R', 'HR', 'RBI', 'AVG', 'SB']].sum(axis=1))
    else:
        return print("Type not recognized")

    # Shift all targets down one year within each player so that each a set of stats are aligned with the next year's results
    df_working = pd.DataFrame()
    for player in df.playerid.unique():
        target = df.loc[df['playerid'] == player]['Target'].shift(-1).copy(deep=True)
        df_working = pd.concat([df_working, target], ignore_index=True)

    # Add shifted target column back to original dataframe.
    df_working.rename(columns={0: 'Target'}, inplace=True)
    df.fillna(0, inplace=True)
    df['Target'] = df_working['Target'].values

    # Drop result columns that have been summed up and the year column
    if type == 'SP':
        df.drop(columns=['W', 'ERA', 'WHIP', 'SO'], inplace=True)
    else:
        df.drop(columns=['R', 'HR', 'RBI', 'SB', 'AVG'], inplace=True)

    return df

In [106]:
def PCA_train(df_in):
    # Create Model
    # Grab model data (Rows with a target value)
    df_model = df_in.dropna(how='any', subset=['Target']).copy(deep=True)
    #df_model.to_csv('df_model.csv')
    df_model.drop(columns=['year'], inplace=True)

    # PCA Components fit and transformed for model
    index_model = df_model.playerid
    df_model.drop(columns='playerid', inplace=True)
    target_model = df_model['Target'].values
    features_model = df_model.drop(columns=['Target']).values
    
    pca = PCA()

    pca_fit = pca.fit(features_model)
    components_model = pca_fit.transform(features_model)

    df_final_model = pd.DataFrame(data=components_model)
    df_final_model = pd.concat([df_final_model, pd.DataFrame(data=target_model, columns=['Target'])], axis=1)
    df_final_model.set_index(index_model, inplace=True)

    #df_final_model.to_csv('df_PCA_model.csv')

    # Regression model trained on model data
    results_model = pd.DataFrame()

    X_model = df_final_model.drop(columns=['Target'])
    y_model = df_final_model['Target']

    regress_model = linear_model.Ridge()
    regress_model.fit(X_model, y_model)

    coef = regress_model.coef_

    print('N: ' + str(df_model.shape[0]))
    print('Intercept: ' + str(regress_model.intercept_))
    print('R2: ' + str(regress_model.score(X_model,y_model)))

    results_model['Feature'] = X_model.columns
    results_model['Coef'] = coef
    results_model.sort_values('Coef', ascending=False, inplace=True)
    #results_model.to_csv('Coefficients_' + type + '.csv')

    return pca_fit, regress_model

def PCA_analysis(pca_fit, regress_model, df_in, df_names, type):
    # Data for test data
    # Create list of players who played in previous year
    lastyear = pd.to_datetime('today').year - 1
    df_currently_playing = df_names.loc[df_names['year'] == str(lastyear)].copy(deep=True)
    print('Currently_playing: ' + str(df_currently_playing.shape[0]))

    # Create test data from only players who played in the previous year
    df_test = df_in.copy(deep=True)
    df_test = df_test.loc[df_test['year'] == lastyear]
    df_test.drop(columns=['year'], inplace=True)

    # Create name list for future adding of names back onto the resultant regression
    df_names_copy = df_names[['Name', 'playerid']].copy(deep=True)

    # Do PCA transform on test data
    index_test = df_test.playerid
    df_test.drop(columns=['playerid'], inplace=True)
    features_test = df_test.drop(columns=['Target']).values
    components_test = pca_fit.transform(features_test)

    df_final_test = pd.DataFrame(data=components_test)
    df_final_test.set_index(index_test, inplace=True)

    X_test = df_final_test

    predictions = pd.DataFrame()
    predictions['playerid'] = df_final_test.index
    predictions['prediction'] = regress_model.predict(X_test)

    predictions = predictions.merge(df_names_copy, how='left', on='playerid')
    predictions = predictions[['playerid', 'Name', 'prediction']].drop_duplicates(subset=['Name'])
    predictions.sort_values(['prediction'], ascending=False, inplace=True)
    predictions.reset_index(drop=True, inplace=True)

    predictions.to_csv('PCA_Results_' + type + '.csv')

    return predictions

In [107]:
def scale_position(df_in):
    df = df_in.copy(deep=True)

    features = df[['prediction']]

    st_scaler = StandardScaler(copy=True, with_mean=False, with_std=True)
    features_scaled = st_scaler.fit_transform(features)

    df['prediction_scaled'] = features_scaled

    return df

In [108]:
def PCA_Weight_Analysis(pca_weights, column_names):
    
    df = pd.DataFrame(columns=['name','weight'])
    
    df['name'] = column_names
    df['weight'] = pca_weights
    
    df.sort_values(by=['weight'], ascending=False, inplace=True)
    
    return df

In [109]:
def process(df_in, df_pos, type):
    df = df_in.copy(deep=True)
    df_out_scaled = pd.DataFrame(columns=['Name', 'prediction', 'prediction_scaled', 'pos'])

    df = clean_data(df, type)
    df = standardize(df)
    if type.find('SP') >= 0:
        df = calc_target(df, 'SP')
        PCA_model_working, regress_model_working = PCA_train(df)
        df = PCA_analysis(PCA_model_working, regress_model_working, df, df_in, 'SP')
    elif type.find('B') >= 0:
        df = calc_target(df, 'B')
        PCA_model_working, regress_model_working = PCA_train(df)
        for p in df_pos.columns:
            if p != 'playerid':
                try:
                    df_out = PCA_analysis(PCA_model_working, regress_model_working, df.loc[df['playerid'].isin(df_pos.loc[df_pos[p] == True, 'playerid'])], df_in, 'B')
                    df_out.loc[df_out['playerid'].isin(df_pos.loc[df_pos[p] == True, 'playerid']), 'pos'] = p
                    #df_out.to_csv('../FB/Positional Results/' + p + '_PCA_Results.csv')
                    df_out_scaled = pd.concat([df_out_scaled, scale_position(df_out)], ignore_index=False)
                    #print(p + ' Finished')
                except:
                    print(p + ' Failed')
            else:
                pass
        df_out_scaled.sort_values('prediction_scaled', ascending=False, inplace=True)
        df_out_scaled.drop_duplicates(subset=['playerid'], inplace=True)
        df_out_scaled.drop(columns=['playerid'], inplace=True)
        #df_out_scaled.to_csv('../FB/Positional Results/PCA_Results_Scaled_by_Position.csv')
    else:
        print('Type not recognized')
        return None

    return None

In [110]:
b_stats = read_files('../FB/Data/Batting', 'years')
sp_stats = read_files('../FB/Data/SP', 'years')
b_stats_skills = read_files('../FB/Data/Batting_Skills', 'years')
sp_stats_skills = read_files('../FB/Data/SP_Skills', 'years')

In [111]:
positions = read_files('../FB/Data/Positions', 'pos')
positions.to_csv('positions.csv')

In [112]:
#process(b_stats, positions, 'B')

In [141]:
b_stats_clean = clean_data(b_stats, 'B')
b_stats_standard = standardize(b_stats_clean)
b_stats_calc = calc_target(b_stats_standard, 'B')

for i in range(0, 3):
    print('Attempt ' + str(i))
    b_stats_working = b_stats_calc[['playerid', 'year', 'Target']]
    b_stats_working = b_stats_working.merge(b_stats_calc.iloc[:, [243] + list(random.sample(range(0, 243), 30))], on='playerid', how='inner')
    b_stats_working.to_csv('test.csv')

    PCA_model_working, regress_model_working = PCA_train(b_stats_working)
    b_stats_working = PCA_analysis(PCA_model_working, regress_model_working, b_stats_working, b_stats, 'B')

Attempt 0
N: 8440
Intercept: 0.9181424989787734
R2: 0.2465332770554064
Currently_playing: 1495
Attempt 1
N: 8440
Intercept: 0.9181424989787735
R2: 0.22692502476333232
Currently_playing: 1495
Attempt 2
N: 8440
Intercept: 0.9181424989787735
R2: 0.24029904344083364
Currently_playing: 1495


In [118]:
b_stats_calc.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2433 entries, 0 to 2432
Data columns (total 246 columns):
 #    Column           Dtype  
---   ------           -----  
 0    G                float64
 1    PA               float64
 2    BB%              float64
 3    K%               float64
 4    ISO              float64
 5    BABIP            float64
 6    OBP              float64
 7    SLG              float64
 8    wOBA             float64
 9    xwOBA            float64
 10   wRC+             float64
 11   BsR              float64
 12   Off              float64
 13   Def              float64
 14   WAR              float64
 15   Age              float64
 16   H                float64
 17   2B               float64
 18   3B               float64
 19   BB               float64
 20   IBB              float64
 21   SO               float64
 22   HBP              float64
 23   SF               float64
 24   SH               float64
 25   GDP              float64
 26   CS               f